# Extracting sequences upstream from differentially expressed genes to identify and count transcription factor binding sites (TFBS)

## Overall purpose of the function
Build a command-line python program to extract upstream sequences from differentially expressed genes to finds motifs or transcription factor binding sites (TFBS) using oPOSSUM. 

Link to website:
http://opossum.cisreg.ca/cgi-bin/oPOSSUM3/opossum_seq_ssa

## What are transcription factors
Transcription factors are proteins that regulate the transcription of genes—that is, their copying into RNA, on the way to making a protein.

https://www.khanacademy.org/science/biology/gene-regulation/gene-regulation-in-eukaryotes/a/eukaryotic-transcription-factors

## How do transcription factors work?

A typical transcription factor binds to DNA at a certain target sequence. Once it's bound, the transcription factor makes it either harder or easier for RNA polymerase to bind to the promoter of the gene.

Some transcription factors **activate transcription**. For instance, they may help the general transcription factors and/or RNA polymerase bind to the promoter, as shown in the diagram below.


In [1]:
from IPython.display import display, Image
Image(url = 'https://ka-perseus-images.s3.amazonaws.com/6567f50d30ad3ac65aff1e815caf202b3abd7111.png')

Other transcription factors **repress transcription**. This repression can work in a variety of ways. As one example, a repressor may get in the way of the basal transcription factors or RNA polymerase, making it so they can't bind to the promoter or begin transcription.

## Binding sites

A typical transcription factor binds to DNA at a certain target sequence (or motif). Once it's bound, the transcription factor makes it either harder or easier for RNA polymerase to bind to the promoter of the gene, and consequently regulates the amount of messenger RNA (mRNA) produced by the gene. Some transcription factors activate transcription, while others repress transcription.
 
Transcription factor binding sites (TFBS) are often located in the 5’-upstream region of target genes to modulate the rate of gene transcription. DNA binding sites can be thus defined as short DNA sequences (typically 4 to 30 base pairs long) that are specifically bound by one or more DNA-binding proteins or protein complexes.

In [2]:
Image(url = 'https://ka-perseus-images.s3.amazonaws.com/1ba8fe2b28b3dd5cd79ec75b74982ee87692dc9e.png')

The flexibility of DNA is what allows transcription factors at distant binding sites to do their job. The DNA loops like cooked spaghetti to bring far-off binding sites and transcription factors close to general transcription factors or "mediator" proteins.

In the cartoon above, an activating transcription factor bound at a far-away site helps RNA polymerase bind to the promoter and start transcribing.

To find and count TFBS my function will extract target genes and background genes in a fasta format by performing the following tasks:
1. Filter differentially expressed genes by logFC values
2. Extract features and coordinates from gff file
3. Extract background sequences from genome
4. Extract target sequences from genome

### 1. Filter differentially expressed genes by logFC values

In [6]:
import pandas as pd
import numpy as np
import math

In [7]:
# Arguments to pass to first function 
filepath1 = '/home/sam079/RNAseq-POMV/Results/ControlvsPOMV6_ALL.csv'
gene_id1 = 'ENTREZID' ## name of column 
threshold1 = 2
threshold_col_id1 = 'logFC'

In [8]:
def get_degenes(filepath, gene_id, threshold, threshold_col_id):
    genes = pd.read_csv(filepath)
    genes = genes.dropna()   
    if genes[gene_id].dtypes == float:
        genes = genes.astype({gene_id:int})
        genes = genes.astype({gene_id:str})
        pass
    elif genes[gene_id].dtypes == int:
        genes = genes.astype({gene_id:str})
    else:
        print("gene names are strings, great!")
    
    DEgenes = genes.loc[(genes[threshold_col_id] >= threshold) | (genes[threshold_col_id] <= -threshold)]
    DEgenes = DEgenes[[gene_id]]
    DEgenes.rename(columns={gene_id:'gene_id'}, inplace=True)
    
    return DEgenes

In [9]:
DEgenes1 = get_degenes(filepath1, gene_id1, threshold1, threshold_col_id1) ## works

### 1. Extract features and coordinates from gff file

https://www.toptal.com/python/comprehensive-introduction-your-genome-scipy

In [14]:
import re

In [15]:
def get_features(gff, feature, search_gff, attribute, coord):
    col_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    mygff = pd.read_csv(gff, sep='\t', comment='#', low_memory=False, header=None, names=col_names)
    CDS = mygff[mygff.type == feature]
    CDS = CDS.copy()

    RE_GENE_NAME = re.compile(r'({}\W)(?P<gene_id>.+?)[,;]'.format(search_gff))
    def extract_gene_name(attributes_str):
        res = RE_GENE_NAME.search(attributes_str)
        if res is None:
            return ''
        else:
            return res.group('gene_id')
    CDS['gene_id'] = CDS.attributes.apply(extract_gene_name)
    
    RE_DESC = re.compile(r'({}\W)(?P<attribute>.+?)[,;]'.format(attribute))
    def extract_description(attributes_str):
        res = RE_DESC.search(attributes_str)
        if res is None:
            return ''
        else:
            return res.group('attribute')
    CDS['attribute'] = CDS.attributes.apply(extract_description)

    CDS.drop('attributes', axis=1, inplace=True)
    
    if coord == 'all':
        CDS_start_points = CDS
    elif coord == 'min':
        CDS_start_points = (CDS.groupby(['seqid', 'gene_id', 'attribute', 'strand'], as_index=False)['start'].min())
    elif coord == 'max':
        CDS_start_points = (CDS.groupby(['seqid', 'gene_id', 'attribute', 'strand'], as_index=False)['start'].max())
    elif coord == 'median':
        CDS_start_points = (CDS.groupby(['seqid', 'gene_id', 'attribute', 'strand'], as_index=False)['start'].median())
        CDS_start_points = CDS_start_points.astype({'start':int})
    else:
        print('Non valid argument given to extraxt gene coordinates for start position')
      
    return CDS_start_points

In [16]:
gff1 = '/OSM/CBR/AF_POMV/work/POMV_RNA_seq/Genomes/Salmo_salar/GCF_000233375.1_ICSASG_v2_genomic.gff'
feature1 = 'CDS' #what feature to extract from the g
search_gff1 = 'GeneID'
attribute1 = 'product'
coord = 'min'

In [17]:
CDS_start_points1 = get_features(gff1, feature1, search_gff1, attribute1, coord)

In [21]:
CDS_start_points1.head

<bound method NDFrame.head of                 seqid    gene_id  \
0         NC_001960.1     808305   
1         NC_001960.1     808306   
2         NC_001960.1     808307   
3         NC_001960.1     808308   
4         NC_001960.1     808309   
5         NC_001960.1     808310   
6         NC_001960.1     808311   
7         NC_001960.1     808312   
8         NC_001960.1     808313   
9         NC_001960.1     808314   
10        NC_001960.1     808315   
11        NC_001960.1     808316   
12        NC_001960.1     808317   
13        NC_027300.1  100136365   
14        NC_027300.1  100136392   
15        NC_027300.1  100136429   
16        NC_027300.1  100136429   
17        NC_027300.1  100136433   
18        NC_027300.1  100136438   
19        NC_027300.1  100136438   
20        NC_027300.1  100136442   
21        NC_027300.1  100136442   
22        NC_027300.1  100136561   
23        NC_027300.1  100136575   
24        NC_027300.1  100136583   
25        NC_027300.1  100136797  

In [12]:
def find_genes(gene_name):
    gene_name = str(gene_name)
    query = CDS_start_points1[CDS_start_points1['ENTREZID'] == gene_name]
    print(query)

In [13]:
find_genes(100380312)

             seqid   ENTREZID                                attribute strand  \
39702  NC_027312.1  100380312   Bone morphogenetic protein 1 precursor      -   
39703  NC_027312.1  100380312  bone morphogenetic protein 1 isoform X1      -   

          start  
39702  85846863  
39703  85846863  


### 3. Extract background sequences from genome

In [20]:
from pyfaidx import Fasta

In [16]:
def create_background_fasta(CDS_start_points1, genome, background_outfile, upstream_nucl):
    genome = Fasta(genome)
    CDS_random = CDS_start_points1.sample(500)
    outfile = open(background_outfile, "w")   
    back_list = []
    back_dict= {}
    
    for index, row in CDS_random.iterrows():
        genes = row['gene_id']
        if row['start'] > upstream_nucl:
            sequences = genome[row['seqid']][row['start'] - upstream_nucl:row['start'] + 3]
        else:
            sequences = genome[row['seqid']][row['start'] - row['start']:row['start']]
        back_dict[genes] = sequences
        back_list.append(back_dict)
        back_dict = {}
    
    for d in back_list:
        for key, value in d.items():
            outfile.write(">" + key + " " + value.fancy_name + "\n" + value.seq + "\n")
    
    outfile.close() 

In [17]:
genome1 = '/OSM/CBR/AF_POMV/work/POMV_RNA_seq/Genomes/Salmo_salar//GCF_000233375.1_ICSASG_v2_genomic.fn'
background_outfile1 = '../Results/background_sequences.txt'
upstream_nucl1 = 5000

In [19]:
#create_background_fasta(CDS_start_points1, gene_id1, genome, background_outfile1, upstream_nucl1)

In [ ]:
def create_target_fasta(DEgenes1, CDS_start_points1, genome, target_outfile, upstream_nucl):
    genome = Fasta(genome)
    outfile = open(target_outfile, "w")
    newdf = pd.merge(DEgenes1, CDS_start_points1)
    seq_list = []
    seq_dict= {}

    for index, row in newdf.iterrows():
        genes = row['gene_id']
        if row['start'] > upstream_nucl:
            sequences = genome[row['seqid']][row['start'] - upstream_nucl:row['start'] + 3]
        else:
            sequences = genome[row['seqid']][row['start'] - row['start']:row['start']]
        seq_dict[genes] = sequences
        seq_list.append(seq_dict)
        seq_dict = {}
    
    for d in seq_list:
        for key, value in d.items():
            outfile.write(">" + key + " " + value.fancy_name + "\n" + value.seq + "\n")
    
    outfile.close() 
    
    return seq_list

In [ ]:
target_outfile1 = '../Results/target_sequences.txt'
upstream_nucl2 = 5000

In [ ]:
#create_target_fasta(DEgenes1, CDS_start_points1, gene_id1, genome, target_outfile1, upstream_nucl2)

# OTHER STUFF

In [143]:
def extract_gene_sequences(gff, feature, search_gff, genome, query_gene):
    col_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    mygff = pd.read_csv(gff, sep='\t', comment='#', low_memory=False, header=None, names=col_names)
    mygff = mygff[mygff.type == feature]
    mygff = mygff.copy()

    RE_GENE_NAME = re.compile(r'({}\W)(?P<gene_id>.+?)[,;]'.format(search_gff))
    def extract_gene_name(attributes_str):
        res = RE_GENE_NAME.search(attributes_str)
        if res is None:
            return ''
        else:
            return res.group('gene_id')
    mygff['gene_id'] = mygff.attributes.apply(extract_gene_name)
    mygff.drop('attributes', axis=1, inplace=True)
    
    genome = Fasta(genome)
    
    mygff_idx = mygff.set_index('gene_id')
    
    if feature == 'gene':
        seqid = mygff_idx.loc[query_gene]['seqid']
        start = mygff_idx.loc[query_gene]['start']
        end = mygff_idx.loc[query_gene]['end']
        sequence = genome[seqid][start:end]
        seq_list = sequence
    
    else:
        genes = mygff_idx.loc[query_gene].index
        seqid = mygff_idx.loc[query_gene]['seqid']
        start = mygff_idx.loc[query_gene]['start']
        end = mygff_idx.loc[query_gene]['end']

        seq_list = []
        seq_dict = {}

        for gene, x, y, z in zip(genes, seqid, start, end):
            sequence = genome[x][y:z]
            seq_dict[gene] = sequence
            seq_list.append(seq_dict)
            seq_dict = {}
    
    return seq_list

In [115]:
genome1 = '/OSM/CBR/AF_POMV/work/POMV_RNA_seq/Genomes/Salmo_salar//GCF_000233375.1_ICSASG_v2_genomic.fna'

In [144]:
extract_gene_sequences(gff1, 'gene', 'GeneID', genome1, '100846970')

>NC_027303.1:69623752-69628218
CACTGTCAATGAAGACCTGAGTGGATATTAACTTTACAAGGAATCACAGAGGAATAACTTCCCGCTCCATCCAATCAAAAGGAAATACCTGCTTTTAGGCTCAATTACAAATTACCGGGATTAAGCGTCATCATAGCTGTAagtgattttattttattattgcaTGCACTGGGCTACGTGGAAACGTTGTGTTTGTGCTAGTGCTCTTTGTACATATTTATAGCAGATATACTTGTAATTTATAGTATATGCTATTTTCTATTCAATTGACTGGGATTTGTTTTTCCATGTGTGCAGTAGCGTGCGTTTTGTCATTGGGCACATATATCCCACATATTGACAGATCTTATTTACCCTTGACACGAGGCTAGAAGCTAAGTGGTGTGCGCGGTGCGCTCATTTGTGAATTTAACGAGACATGGAACAATGCAGTTATAATTAATGCAGCTTTGCAGTAGTATATTTATGACATTTCAAGAGTTAGTTAAAATGTAAAGTTGCAAGTTTTAGTGGGTCAAGGGTTGCAACACTGTAATTTAGAGGCACTGTAATGCATTGGTCAAAATATGCATGCGCTTGTTCAATGCACGATTAAAGTGTGCGTGGGTGAATACGGCTGTTCGTTCTTTTCAAGAGCTATAGAACCCCGCTCTCTTTAACCTCGAATGCAACATAAATAGCGATATAGCAGCAGTggcggaaaaagtacccaattgtcatacttgactaaaagtaaagataccttaatagaaaatgacccaAGTAAAGAGTTacccaataaaatactacttgagtaaaagcccaaaagtatttggttttaaatatacttaagtatcaaaagtacaagtataaatcatttcatattccttatattaagcaaaccagatggcaccattatattgttttttatttatttatgaatagccactccaacactcattagcatcatttacaaacgtgtttagtgag

In [132]:
col_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
mygff = pd.read_csv(gff1, sep='\t', comment='#', low_memory=False, header=None, names=col_names)
mygff = mygff[mygff.type == 'CDS']
mygff = mygff.copy()

RE_GENE_NAME = re.compile(r'({}\W)(?P<gene_id>.+?)[,;]'.format('GeneID'))
def extract_gene_name(attributes_str):
    res = RE_GENE_NAME.search(attributes_str)
    if res is None:
        return ''
    else:
        return res.group('gene_id')


    

mygff['gene_id'] = mygff.attributes.apply(extract_gene_name)
mygff.drop('attributes', axis=1, inplace=True)
mygff_idx = mygff.set_index('gene_id')

In [133]:
query_gene = '100196051'

In [134]:
mygff_idx.loc[query_gene]

,seqid,source,type,start,end,score,strand,phase
gene_id,,,,,,,,
100196051,NC_027320.1,BestRefSeq,CDS,17643150,17643189,.,-,0
100196051,NC_027320.1,BestRefSeq,CDS,17642541,17642664,.,-,2
100196051,NC_027320.1,BestRefSeq,CDS,17641533,17641708,.,-,1
100196051,NC_027320.1,BestRefSeq,CDS,17641137,17641300,.,-,2
100196051,NC_027320.1,BestRefSeq,CDS,17640003,17640162,.,-,0
100196051,NC_027320.1,BestRefSeq,CDS,17639714,17639829,.,-,2
100196051,NC_027320.1,BestRefSeq,CDS,17635572,17635601,.,-,0


In [137]:
genes = mygff_idx.loc[query_gene].index

seqid = mygff_idx.loc[query_gene]['seqid']
start = mygff_idx.loc[query_gene]['start']
end = mygff_idx.loc[query_gene]['end']

genome = Fasta(genome1)

seq_list = []
seq_dict = {}

for gene, x, y, z in zip(genes, seqid, start, end):
    sequence = genome[x][y:z]
    seq_dict[gene] = sequence
    seq_list.append(seq_dict)
    seq_dict = {}

SyntaxError: 'return' outside function (<ipython-input-137-38bb5a777501>, line 18)

### 3. Find and count transcription factor binding sites (TFBSs) in the extracted sequences

https://biopython.readthedocs.io/en/latest/Tutorial/chapter_motifs.html

In [39]:
from Bio import motifs

In [49]:
m1 = motifs.read(open("../Data/MA0051.1.sites"), "sites")

In [50]:
m1.counts

{'G': [7, 10, 0, 0, 0, 10, 0, 12, 0, 0, 0, 6, 2, 3, 2, 1, 1, 1],
 'A': [0, 2, 12, 11, 12, 2, 0, 0, 12, 12, 12, 0, 0, 5, 6, 6, 5, 3],
 'T': [1, 0, 0, 1, 0, 0, 6, 0, 0, 0, 0, 0, 3, 2, 4, 4, 4, 2],
 'C': [4, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 6, 7, 2, 0, 1, 2, 6]}

In [51]:
count = 0
for d in seq_list:
    for key, value in d.items():
        for pos, motif in m1.instances.search(d[key]):
            if value.fancy_name == value.fancy_name:
                count =+ 1
            print(value.fancy_name, key, pos, motif, count)